<a href="https://colab.research.google.com/github/galua2001/colab/blob/main/curve1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
using System.Collections.Generic;
using UnityEngine;

public class Draw : MonoBehaviour
{
    [Header("공유 곡선 (0~1 정규화된 로컬 좌표)")]
    public static List<Vector2> normalizedPoints = new List<Vector2>(); // (0~1) 범위
    public Color curveColor = Color.red;

    [Header("카메라 연결")]
    public Camera mainCamera;

    private static List<Draw> instances = new List<Draw>();
    private LineRenderer line;

    void Awake()
    {
        if (!instances.Contains(this))
            instances.Add(this);

        SetupLineRenderer();
    }

    void Start()
    {
        DrawCurve();
    }

    void OnDestroy()
    {
        if (instances.Contains(this))
            instances.Remove(this);
    }

    void Update()
    {
        if (mainCamera == null) return;

        // 마우스 왼쪽 클릭 → 정규화된 로컬 좌표로 저장
        if (Input.GetMouseButtonDown(0))
        {
            Vector2 mouseWorld = mainCamera.ScreenToWorldPoint(Input.mousePosition);
            Vector2 local = transform.InverseTransformPoint(mouseWorld);
            Vector2 normalized = ToNormalized(local);

            if (IsMainController())
            {
                normalizedPoints.Add(normalized);
                UpdateAllCurves();
            }
        }

        // C 키 → 초기화
        if (Input.GetKeyDown(KeyCode.C) && IsMainController())
        {
            normalizedPoints.Clear();
            UpdateAllCurves();
        }

        // 마우스 오른쪽 클릭 → 곡선 조작
        if (Input.GetMouseButtonDown(1))
        {
            Vector2 mouseWorld = mainCamera.ScreenToWorldPoint(Input.mousePosition);
            Vector2 local = transform.InverseTransformPoint(mouseWorld);

            bool isLeft = local.x < 0;
            bool isTop = local.y > 0;

            if (isLeft && isTop) Rotate180();
            else if (!isLeft && isTop) MirrorY();
            else if (isLeft && !isTop) ClearCurve();
            else MirrorDiagonal();

            UpdateAllCurves();
        }
    }

    void SetupLineRenderer()
    {
        line = GetComponent<LineRenderer>();
        if (line == null)
            line = gameObject.AddComponent<LineRenderer>();

        line.startWidth = 0.05f;
        line.endWidth = 0.05f;
        line.material = new Material(Shader.Find("Sprites/Default"));
        line.sortingOrder = 1;
        line.startColor = curveColor;
        line.endColor = curveColor;
        line.useWorldSpace = false; // 로컬 기준에서 그림
    }

    void DrawCurve()
    {
        if (normalizedPoints.Count < 2)
        {
            line.positionCount = 0;
            return;
        }

        line.positionCount = normalizedPoints.Count;

        for (int i = 0; i < normalizedPoints.Count; i++)
        {
            Vector2 local = FromNormalized(normalizedPoints[i]);
            line.SetPosition(i, local);
        }
    }

    void UpdateAllCurves()
    {
        foreach (var instance in instances)
        {
            instance.DrawCurve();
        }
    }

    // (0~1)로 정규화
    Vector2 ToNormalized(Vector2 localPos)
    {
        Vector2 size = GetSize();
        return new Vector2((localPos.x + size.x / 2f) / size.x, (localPos.y + size.y / 2f) / size.y);
    }

    // 정규화 → 실제 로컬 위치로 환산
    Vector2 FromNormalized(Vector2 norm)
    {
        Vector2 size = GetSize();
        return new Vector2(norm.x * size.x - size.x / 2f, norm.y * size.y - size.y / 2f);
    }

    Vector2 GetSize()
    {
        var sprite = GetComponent<SpriteRenderer>();
        return sprite != null ? sprite.bounds.size : Vector2.one;
    }

    // 조작 기능들
    void Rotate180()
    {
        for (int i = 0; i < normalizedPoints.Count; i++)
        {
            normalizedPoints[i] = -normalizedPoints[i] + Vector2.one;
        }
    }

    void MirrorY()
    {
        for (int i = 0; i < normalizedPoints.Count; i++)
        {
            Vector2 p = normalizedPoints[i];
            normalizedPoints[i] = new Vector2(p.x, 1f - p.y);
        }
    }

    void MirrorDiagonal()
    {
        for (int i = 0; i < normalizedPoints.Count; i++)
        {
            Vector2 p = normalizedPoints[i];
            normalizedPoints[i] = new Vector2(p.y, p.x);
        }
    }

    void ClearCurve()
    {
        normalizedPoints.Clear();
    }

    bool IsMainController()
    {
        return instances.Count > 0 && instances[0] == this;
    }
}